In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
# import dummy classifier
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.inspection import permutation_importance
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix


In [ ]:
df_meal = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/meal/combined_features.csv', index_col=0)
df_meal = df_meal.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

df_play = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/play/combined_features.csv', index_col=0)
df_play = df_play.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

In [ ]:
# Map 'no_rad' to 0 and 'rad' to 1
df_meal['label'] = df_meal['label'].map({'no_rad': 0, 'rad': 1})
df_play['label'] = df_play['label'].map({'no_rad': 0, 'rad': 1})



In [ ]:
# Reset the index
df_meal = df_meal.reset_index()

# Create the 'group' column and group by it
df_meal['group'] = df_meal['s_id'].str.split('_').str[0].astype(int)
df_grouped_meal = df_meal.drop(columns=['s_id']).groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
#df_grouped_meal['s_id'] = df_meal.groupby('group')['s_id'].first()
df_grouped_meal.index.name = 's_id'

# Reset the index
df_play = df_play.reset_index()

# Create the 'group' column and group by it
df_play['group'] = df_play['s_id'].str.split('_').str[0].astype(int)
df_grouped_play = df_play.drop(columns=['s_id']).groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
df_grouped_meal.index.name = 's_id'
df_grouped_play.index.name = 's_id'


# Reset the index of the grouped dataframes
#df_grouped_meal = df_grouped_meal.reset_index()
#df_grouped_play = df_grouped_play.reset_index()

# Add a new 'group' column to each DataFrame
df_grouped_meal['group'] = 'meal'
df_grouped_play['group'] = 'play'



In [ ]:

# Concatenate the DataFrames
final_df = pd.concat([df_grouped_meal, df_grouped_play], axis=0)

final_df

In [ ]:
final_df = final_df.drop(columns=['index'])
final_df.set_index('s_id', inplace=True)
final_df


In [ ]:
final_df.set_index('group', append=True, inplace=True)

In [ ]:
final_df

In [ ]:
# Reset the index
final_df.reset_index(inplace=True)

# Convert the first level of the index to integers
final_df.iloc[:, 0] = final_df.iloc[:, 0].astype(int)

# Set the index again
final_df.set_index(final_df.columns[:2].tolist(), inplace=True)
final_df

In [ ]:
speech = pd.read_excel('/Users/andrei-macpro/Documents/Data/classification/speech/classification.xlsx', index_col=0)

In [ ]:
# set index name to s_id
speech.index.name = 's_id'
# remove age column from speech
speech = speech.drop(columns=['age'])

In [ ]:
# Create a boolean mask where True indicates a duplicated index
mask = speech.index.duplicated(keep='first')

# Use np.where to assign 'meal' to the first occurrence and 'play' to the second
speech['group'] = np.where(mask, 'play', 'meal')

In [ ]:
speech.set_index('group', append=True, inplace=True)

In [ ]:
## Separate additions to speech features 

to_add = final_df[['child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max', 
                   'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',]]
speech[to_add.columns] = to_add

In [ ]:
speech['label'] = speech['label'].map({'no_rad': 0, 'rad': 1})


In [ ]:
speech = speech.reset_index()
# set s_id as index for final_df
speech.set_index('s_id', inplace=True)
speech

In [ ]:
# drop nan rows from speech
speech = speech.dropna()

In [ ]:
speech_play = speech[speech['group'] == 'play']

In [ ]:


# Perform a grid search for each classifier
X = speech_play[['Proportion speech child']]
y = speech_play['label']
groups = speech_meal.index

# Create a GroupKFold object
gkf = KFold(n_splits=5)

# Define the classifiers and their parameters
classifiers = [
('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {'lr__C': [0.01, 0.1, 1, 10, 100], 'lr__penalty': ['l1', 'l2'], 'lr__solver': ['liblinear', 'saga']}),
    ('svc_linear', SVC(kernel='linear'), {'svc_linear__C': [0.01, 0.1, 1, 10, 100]}),
    ('svc_rbf', SVC(kernel='rbf'), {'svc_rbf__C': [0.01, 0.1, 1, 10, 100], 'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]}),
    ('rf', RandomForestClassifier(), {'rf__n_estimators': [10, 50, 100, 200], 'rf__max_depth': [None, 5, 10, 15], 'rf__min_samples_split': [2, 5, 10]})
]
    

import pandas as pd

# Initialize a list to store the results
results = []

# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled = shuffle(X, y,  random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([('scaler', StandardScaler()), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro',n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro', n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy',  n_jobs=-1)

        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'best_score': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean()
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').std()

In [ ]:
final_df.columns

In [ ]:
final_df = final_df.reset_index()
# set s_id as index for final_df
final_df.set_index('s_id', inplace=True)

In [ ]:
final_df

In [ ]:
df_pca

In [ ]:


final_df = final_df.reset_index()
final_df.set_index(['group', 's_id'], inplace=True)
speech = speech.reset_index() 
speech.set_index(['group', 's_id'], inplace=True)
# drop label column from final_df
final_df = final_df.drop(columns=['label'])
multimodal = pd.concat([speech, final_df], axis=1)

In [ ]:
multimodal

In [ ]:
multimodal = multimodal.dropna()

In [ ]:
speech = speech.dropna()

In [ ]:
speech = speech.reset_index()
# set s_id as index for final_df
speech.set_index('s_id', inplace=True)
speech

In [ ]:
speech.columns

In [ ]:
X

In [ ]:
multimodal = multimodal.reset_index()
multimodal.set_index(['s_id'], inplace=True)
multimodal

In [ ]:
multimodal['group'] = multimodal['group'].map({'play': 0, 'meal': 1})

In [ ]:
X

In [ ]:
# Custom transformer to apply PCA to a subset of feature

X = multimodal[['Proportion speech child',
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]

subset_pca = ['cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max', 
            'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']


#X = X[['Proportion speech child', 'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]
y = multimodal['label']
groups = multimodal.index

remaining_features = [feat for feat in X.columns if feat not in subset_pca]

# Create a ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('pca_pipeline', Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA())
    ]), subset_pca),
    ('remaining', StandardScaler(), remaining_features)
])

# Create a GroupKFold object
gkf = GroupKFold(n_splits=5)
best_models = {}
conf_matrix_sum = np.zeros((2, 2))
# Define the classifiers and their parameters
classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear'), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf'), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'preprocessor__pca_pipeline__pca__n_components': [2, 4],
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    })
]
    

import pandas as pd

# Initialize a list to store the results
results = []

# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled, groups_shuffled = shuffle(X, y, groups, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([
                              ('preprocessor', preprocessor),(name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled, groups=groups_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', groups=groups_shuffled,n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', groups=groups_shuffled, n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro', groups=groups_shuffled, n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy', groups=groups_shuffled, n_jobs=-1)
        best_pca_components = grid_search.best_params_.get('preprocessor__pca_pipeline__pca__n_components', None)

        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'best_score': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': best_pca_components
        })
        if name not in best_models or grid_search.best_score_ > best_models[name]['best_score']:
                best_models[name] = {
                    'best_estimator': grid_search.best_estimator_,
                    'best_params': grid_search.best_params_,
                    'best_score': grid_search.best_score_,
                    'random_state': i
                }

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)



In [ ]:
best_models['lr']['best_estimator']

In [ ]:
import seaborn as sns
# Visualize the average confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(average_conf_matrix, annot=True, fmt=".2f", cmap="Blues", xticklabels=['no rad', 'rad'], yticklabels=['no rad', 'rad'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Average Confusion Matrix')
plt.show()

In [ ]:
results_df.groupby('classifier').std()

In [ ]:
result = permutation_importance(best_models['svc_linear']['best_estimator'], X, y, n_repeats=30, random_state=0, scoring='accuracy')


In [ ]:
feature_importances_combined = pd.DataFrame({
    'Feature': X.columns,
    'Importance': np.abs(result['importances_mean'])
})

feature_importances_combined = feature_importances_combined.sort_values(by='Importance', ascending=False)
# Plot the feature importances
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importances_combined, palette='Set1')
plt.title('Feature Importances Meal + Play')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

## Best model for speech child + movement PCA (73.33% acc) : {'preprocessor__pca_pipeline__pca__n_components': 2, 'xgb__learning_rate': 0.01, 'xgb__max_depth': 5, 'xgb__n_estimators': 200}

In [ ]:
results_df

In [ ]:
dfspeech

In [ ]:
final_df = final_df.reset_index()
final_df.set_index(['s_id'], inplace=True)
final_df

In [ ]:
final_df.columns

In [ ]:


# Perform a grid search for each classifier
X = final_df[[
       'cg_movement_mean', 'cg_movement_var', 'cg_movement_min', 'cg_movement_max',
       'child_movement_mean', 'child_movement_var', 'child_movement_min', 'child_movement_max']]
y = final_df['label']
groups = final_df.index

# Create a GroupKFold object
gkf = GroupKFold(n_splits=5)

# Define the classifiers and their parameters
classifiers = [
    ('dummy', DummyClassifier(strategy='most_frequent'), {}),
    ('lr', LogisticRegression(), {
        'pca__n_components': [2, 4],
        'lr__C': [0.01, 0.1, 1, 10, 100],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    }),
    ('svc_linear', SVC(kernel='linear'), {
        'pca__n_components': [2, 4],
        'svc_linear__C': [0.01, 0.1, 1, 10, 100]
    }),
    ('svc_rbf', SVC(kernel='rbf'), {
        'pca__n_components': [2, 4],
        'svc_rbf__C': [0.01, 0.1, 1, 10, 100],
        'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]
    }),
    ('rf', RandomForestClassifier(), {
        'pca__n_components': [2, 4],
        'rf__n_estimators': [10, 50, 100, 200],
        'rf__max_depth': [None, 5, 10, 15],
        'rf__min_samples_split': [2, 5, 10]
    }),
    ('et', ExtraTreesClassifier(), {
        'pca__n_components': [2, 4],
        'et__n_estimators': [50, 100, 200],
        'et__max_depth': [None, 5, 10, 20],
        'et__min_samples_split': [2, 5, 10]
    }),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'pca__n_components': [2, 4],
        'xgb__n_estimators': [50, 100, 200],
        'xgb__learning_rate': [0.01, 0.1, 0.2],
        'xgb__max_depth': [3, 5, 10]
    })
]
    
    

import pandas as pd

# Initialize a list to store the results
results = []

# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled, groups_shuffled = shuffle(X, y, groups, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([('scaler', StandardScaler()),
                             ('pca', PCA()), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf, n_jobs=-1)
        grid_search.fit(X_shuffled, y_shuffled, groups=groups_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', groups=groups_shuffled,n_jobs=-1)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', groups=groups_shuffled, n_jobs=-1)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro', groups=groups_shuffled, n_jobs=-1)
        accuracy_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='accuracy', groups=groups_shuffled, n_jobs=-1)
         # Extract the number of PCA components from the best parameters
        best_pca_components = grid_search.best_params_.get('pca__n_components', None)
        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'best_score': accuracy_scores.mean(),
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean(),
            'pca_components': best_pca_components
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').mean()

In [ ]:
speech